In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_data(path):
    df = pd.read_csv(path)
    return df

In [ ]:
aes_ni = import_data('../../results/crypto/aes-ni/processed.csv')
aes_noni = import_data('../../results/crypto/aes-noni/processed.csv')
chacha20 = import_data('../../results/crypto/chacha20/processed.csv')

In [ ]:
aes_noni['crypto'] = 'aes-noni'
aes_ni['crypto'] = 'aes-ni'
chacha20['crypto'] = 'chacha'

cryptos = {
    'aes-noni': aes_noni,
    'aes-ni': aes_ni,
    'chacha': chacha20
}

for k, df in cryptos.items():
    df['server'] = df['server'].apply(lambda x: x.rstrip('-aes'))
    df['client'] = df['client'].apply(lambda x: x.rstrip('-aes'))

data = pd.concat(cryptos.values(), ignore_index=True)

In [ ]:
def plot_goodput(df):

    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    ciphers = df.crypto.unique()

    implementation_order = {
        'lsquic': 1,
        'quiche': 2,
        'quich': 2,
        'tcp-tls': 3,
        'tls1.3': 4,
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    print(df.groupby(by=['server', 'crypto'], as_index=False).agg({'goodput': ['mean', 'std']}))

    
    cipher_names = {
        'aes-noni': "AES w/o",
        'aes-ni': "AES-NI",
        'chacha': "ChaCha20"
    }
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'quiche': 'quiche',
        'quich': 'quiche',
        'tls1.3': 'TCP/TLS',
    }
        
    colors = [ColorDarkerBlue, ColorGreen, ColorRed]
    patterns = [" "] * len(colors)
    linestyles = ['-'] * len(colors)
    markers = ['^', 'o', 's']
    
    def style(i):
        return {
            #"boxprops": dict(facecolor=c, color=c),
            "showmeans": True,
            "boxprops": dict(color=colors[i], linestyle=linestyles[i], facecolor="white"),
            "capprops": dict(color=colors[i], alpha=0),
            "whiskerprops": dict(alpha=0),
            "flierprops": dict(color=colors[i], markeredgecolor=colors[i]),
            #"medianprops": dict(color=c),
            #"meanprops": dict(color=c),
            "medianprops": dict(color=colors[i]),
            "meanprops": dict(markerfacecolor=colors[i], markeredgecolor=colors[i], marker=markers[i]),
            "whis": [5,95],
            "showfliers": False,
            "widths": .22,
            "positions": [x - 0.3 + 0.3 * i for x in range(len(ciphers))],
            "patch_artist": True,
        }
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    plot_shades(ciphers, ax1, 0.5)
    
    for idx, imp in enumerate(implementations):
        
        d = df.loc[(df.server == imp) & (df.client == imp)]
        bp1 = ax1.boxplot([d[d.crypto == c].goodput for c in ciphers], labels=ciphers,
            #hatch=patterns[0],
            **style(idx),
           )
    
    ax1.autoscale()
    
    ax1.set(xticks=range(len(ciphers)), xticklabels=[cipher_names[c] for c in ciphers])
    
    ax1.set_ylabel('Goodput [Gbit/s]')
    ax1.set_ylim(0, 9000)
    
    mkfunc = lambda x, pos: f'{(x * 1e-3):.0f}'
    mkformatter = matplotlib.ticker.FuncFormatter(mkfunc)
    ax1.yaxis.set_major_formatter(mkformatter)
    
    legend_elements = [
        #Patch(facecolor='white', edgecolor=colors[i], linestyle=linestyles[i], label=implementation_names[imp])
        mlines.Line2D([], [], color=colors[i], marker=markers[i], linestyle='None', label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    ax1.legend(handles=legend_elements, loc="upper left", facecolor='white', framealpha=1, ncol=1)
    fig.tight_layout()
    fig.savefig('crypto.pdf')

    

In [ ]:
plot_goodput(data)